In [2]:
import time
import pandas as pd
import numpy as np
import re
import glob
import torch
import itertools
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from numpy import zeros
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
data2=pd.read_csv('news.csv')
data2.head()

C:\Users\Lakys\AppData\Local\Temp\ipykernel_1864\1704981109.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  data2=pd.read_csv('news.csv')


,Unnamed: 0,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data2=pd.concat([pd.Series(data2["text"], name="text"), pd.Series(data2["label"], name="label")], axis=1)
data2.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [5]:
len(data2)

7795

In [6]:
data3=data2[(data2["label"]=="FAKE") | (data2["label"]=="REAL")]

In [7]:
set(list(data3["label"]))

{'FAKE', 'REAL'}

In [8]:
data3["label"].value_counts()

label
REAL    3161
FAKE    3154
Name: count, dtype: int64

In [9]:
le = preprocessing.LabelEncoder()
labe=data3["label"]

labels_encoded=le.fit(labe)
labels_=le.transform(labe)

In [10]:
list(zip(data3["label"][:5], labels_[:5]))

[('FAKE', 0), ('FAKE', 0), ('REAL', 1), ('FAKE', 0), ('REAL', 1)]

In [11]:
set(labels_)

{0, 1}

In [12]:
text_for_this=[]
for i in data3["text"]:
    text_for_this.append(' '.join(re.findall(r'[a-zA-Z]+', str(i))))

In [13]:
#bytestring_=[i.encode() for i in text_for_this]

In [14]:
datalist1=glob.glob('glove_file_*')

In [15]:
combined_datalist=[pd.read_csv(i) for i in datalist1]
words_=list(itertools.chain.from_iterable([list(i.word) for i in combined_datalist]))
values_=list(itertools.chain.from_iterable([list(i.values_) for i in combined_datalist]))
values_=[np.array(re.findall(r'[\d\.]{1,8}',str(i)),dtype='float32') for i in values_]
values_2=[i[:100] for i in values_]
embed_index=dict(zip(words_,values_2))

In [16]:
word_tokenizer = Tokenizer()

In [17]:
word_tokenizer.fit_on_texts(text_for_this)

In [18]:
embedded_skill=word_tokenizer.texts_to_sequences(text_for_this)

In [19]:
vocab_length = len(word_tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embed_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

len_sent=list([len(i) for i in embedded_skill])
length_long_sentence=max(len_sent)

padded_sentences = pad_sequences(embedded_skill, length_long_sentence, padding='post')

validation_split=.2
indices=np.arange(np.array(padded_sentences,dtype=object).shape[0])
np.random.shuffle(indices)   

data_rand=padded_sentences[indices]

In [20]:
labels_rand=np.array(labels_)[indices]

In [21]:
val_sample=int(validation_split * data3.shape[0])

In [22]:
X_train=data_rand[:-val_sample]
y_train=labels_rand[:-val_sample]
X_test=data_rand[-val_sample:]
y_test=labels_rand[-val_sample:]

In [23]:
x_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
x_cv = torch.tensor(X_test, dtype=torch.long)
y_cv = torch.tensor(y_test, dtype=torch.long)

train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

In [24]:
class BiLSTM(nn.Module):
  def __init__(self):
    super(BiLSTM, self).__init__()
    self.hidden_size = 5
    drp = 0.4
    n_classes = len(le.classes_)
    self.embedding = nn.Embedding(max_features, embed_size)
    self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
    self.embedding.weight.requires_grad = True
    self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.linear = nn.Linear(self.hidden_size*4 , 1, bias=False)
    self.relu = nn.ReLU()
    self.out = nn.Linear(1, n_classes, bias=False)


  def forward(self, x):
    h_embedding = self.embedding(x)
    h_lstm, _ = self.lstm(h_embedding)
    avg_pool = torch.mean(h_lstm, 1)
    max_pool, _ = torch.max(h_lstm, 1)
    conc = torch.cat(( avg_pool, max_pool), 1)
    conc = self.relu(self.linear(conc))
    out = self.out(conc)
    return out

In [25]:
embed_size=100
max_features=vocab_length
n_epochs = 11
model = BiLSTM()
loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=.0001)
model.cpu()

BiLSTM(
  (embedding): Embedding(63464, 100)
  (lstm): LSTM(100, 5, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=20, out_features=1, bias=False)
  (relu): ReLU()
  (out): Linear(in_features=1, out_features=2, bias=False)
)

In [26]:
output=[]
val_preds_=[]
for epoch in range(n_epochs):
    start_time = time.time()
    
    model.train()
    
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):        
      y_pred = model(x_batch)      
      loss = loss_fn(y_pred, y_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      avg_loss += loss.item() / len(train_loader)
      
    
    avg_val_loss = 0.
    val_preds=[]
    val_preds_.append(val_preds)
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        
      y_pred = model(x_batch).detach()
      avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
      val_preds.append(F.sigmoid(y_pred).cpu().numpy())

    elapsed_time = time.time() - start_time

    print(f'epoch:{epoch}, average validation loss:{avg_val_loss}, elapsed time:{elapsed_time}')
    output.append('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))

epoch:0, average validation loss:0.682505376636982, elapsed time:74.93276000022888
epoch:1, average validation loss:0.6029398962855339, elapsed time:70.3253698348999
epoch:2, average validation loss:0.5518989183008671, elapsed time:68.00787401199341
epoch:3, average validation loss:0.5155128121376036, elapsed time:66.31374740600586
epoch:4, average validation loss:0.5111503198742866, elapsed time:66.59367680549622
epoch:5, average validation loss:0.5058595940470696, elapsed time:67.01465582847595
epoch:6, average validation loss:0.4965121529996395, elapsed time:66.36403131484985
epoch:7, average validation loss:0.4982491992413997, elapsed time:66.68814969062805
epoch:8, average validation loss:0.5293045863509178, elapsed time:66.83333706855774
epoch:9, average validation loss:0.4955587893724441, elapsed time:67.79288172721863
epoch:10, average validation loss:0.4963989086449146, elapsed time:67.99239325523376


In [27]:
val_accuracy=[]
for i in range(len(val_preds_)):
    val_cov_arr=[]
    for p in range(len(val_preds_[i])):
        val_cov_=[]
        length=len(val_cov_)
        for s in val_preds_[i][p]:
            val_cov=[]
            for l in s:
                if l>.6:
                    val_cov.append(int(0))
                else:
                    val_cov.append(int(1))
            val_cov_.append(val_cov[np.array(val_cov).argmax()])
        val_cov_arr.append(np.sum(np.array(val_cov_))/len(val_cov_))
    val_accuracy.append(np.mean(val_cov_arr))        

In [28]:
output_df=pd.DataFrame([i.split('\t') for i in output])
output_df["validation_accuracy"]=[round(i,4) for i in val_accuracy]
output_df.columns=['epoch','loss','validation_loss','time','validation_accuracy']
output_df

,epoch,loss,validation_loss,time,validation_accuracy
0,Epoch 1/11,loss=0.6894,val_loss=0.6825,time=74.93s,1.0
1,Epoch 2/11,loss=0.6437,val_loss=0.6029,time=70.33s,1.0
2,Epoch 3/11,loss=0.5409,val_loss=0.5519,time=68.01s,1.0
3,Epoch 4/11,loss=0.4791,val_loss=0.5155,time=66.31s,1.0
4,Epoch 5/11,loss=0.4297,val_loss=0.5112,time=66.59s,1.0
5,Epoch 6/11,loss=0.4029,val_loss=0.5059,time=67.01s,1.0
6,Epoch 7/11,loss=0.3842,val_loss=0.4965,time=66.36s,1.0
7,Epoch 8/11,loss=0.3776,val_loss=0.4982,time=66.69s,1.0
8,Epoch 9/11,loss=0.3700,val_loss=0.5293,time=66.83s,1.0
9,Epoch 10/11,loss=0.3641,val_loss=0.4956,time=67.79s,1.0


In [29]:
test_data=pd.read_csv("test.csv", encoding="latin1")

In [30]:
text_for_test=[]
for i in test_data["text"]:
    text_for_test.append(' '.join(re.findall(r'[a-zA-Z]+', str(i))))
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(text_for_test)
embedded_skill=word_tokenizer.texts_to_sequences(text_for_test)

vocab_length = len(word_tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embed_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

len_sent=list([len(i) for i in embedded_skill])
length_long_sentence=max(len_sent)

padded_sentences_ = pad_sequences(embedded_skill, length_long_sentence, padding='post')

In [31]:
test_data1 = torch.tensor(padded_sentences_, dtype=torch.long)
test_data_ = torch.utils.data.TensorDataset(test_data1)
test_data_loader = torch.utils.data.DataLoader(test_data_, batch_size=32, shuffle=True)

In [32]:
pred = []
model.eval()

for inputs in test_data_loader:
    y_pred = model(inputs[0]).detach()
    pred.append(y_pred)

In [33]:
prediction_class=[]
for i in range(len(pred)):
    
    for p in range(len(pred[i])):
        val_cov_=[]
        length=len(val_cov_)
        for s in val_preds_[i][p]:
            val_cov=[]
            
            for l in s:
                if l>.6:
                    val_cov.append(int(0))
                else:
                    val_cov.append(int(1))
        val_cov_.append(val_cov[np.array(val_cov).argmax()])
        prediction_class.append(val_cov_)

In [34]:
classification_dictionary={1:"REAL", 0:"FAKE"}

In [35]:
prediction_class=[classification_dictionary[i[0]] for i in prediction_class]

In [36]:
test_data["Prediction Class"]=prediction_class

In [37]:
test_data

,text,Prediction Class
0,A roundup of some of the most popular but comp...,REAL
1,The Biden administration is working on a new d...,REAL
2,"Pope Francis, in the first-ever papal address ...",REAL
